# Linking Dependencies

In [ ]:
# imports
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from bayes_opt.bayesian_optimization import BayesianOptimization
from bayes_opt.helpers import UtilityFunction
from sklearn.gaussian_process.kernels import RBF

__all__ = ["BayesianOptimization", "UtilityFunction"]
import warnings
warnings.filterwarnings("ignore")

# Data Input

In [ ]:
# read excel file
raceDataFrame = pd.read_excel('data.xlsx', sheetname='Sheet1')
raceData = np.array(raceDataFrame.values)
#raceData[np.argsort(raceData[:, -1])]
numberOfRaces = int(raceData[-1][-1])
proportion = 1/3
separationPoint = int(proportion * (numberOfRaces - 1))

# init array containing all races
#(each race is a sequence of horse profile(X) and speed(Y), with speed as the first index)
races = []
for _ in range (0, numberOfRaces):
    races.append([])

# separate data into X and Y, indexed by races (each as an array of horse-in-race entry)
for i in range (0, len(raceData)):
    # group into races
    currentRow = raceData[i]
    raceNumber = int(currentRow[-1])
    rowData = currentRow[0:-1]
    races[raceNumber-1].append(rowData)

# convert races into np.arrays
races = list(map(lambda x: np.array(x), races))
    
# split data into different sets
np.random.shuffle(races)
historicalRaces = races[:separationPoint]
futureRaces = races[separationPoint:]

# all races ready as an array: races (randomised order)
# historical races are used to initialise the GP model within the Bayesian Optimiser
# future races are used to simulate the selection performed by the Bayesian Optimiser

# Use Modified Bayesian Optimisation

In [ ]:
parameter_bounds = {'win': (0, 1000),
                    'horse_rating': (0, 300),
                    'horse_weight': (300, 800),
                    'handicapped_weight': (0, 100),
                    'carried_weight': (0, 100),
                    'lane_number': (1, 18),
                    'running_one': (1, 18),
                    'running_two': (1, 18),
                    'placing': (1, 18),
                    'length_behind_winner': (0, 100),
                   }

# create and init a new bayesian optimiser that follows the param limits specified above
bayesianOptimiser = BayesianOptimization(parameter_bounds, gp_kernel = RBF())

## Define Kernel, Kappa, Acquisition Function, Batch Size, etc.

In [ ]:
acquisitionFunctionFlag = 'ei' # poi (probability of improvement) | ucb (upper confidence bound) | ei (expected improvement)
kappa = 2.576
xi = 0.02
theta0 = 0.02
nugget = 0.01

bayesianOptimiser.setup(acquisitionFunctionFlag, kappa, xi)

## Initialise GP Model with Historical Data

In [ ]:
# strip historical races
historicalHorses = []

for race in historicalRaces:
    for horse in race:
        historicalHorses.append(horse)
        
bayesianOptimiser.initialize(np.vstack(historicalHorses))

## Iterate for n Rounds (Races Allowed to Observe)

In [ ]:
# can only select from n sets of races, each time picking 1 race to watch
# after watching n races, predict for a race and place a bet ... see if successful ...
splitRatio = 0.8
splitPoint = int(splitRatio * (len(futureRaces) - 1))
np.random.shuffle(futureRaces)
trainRaces = futureRaces[:splitPoint]
testRaces = np.array(futureRaces[splitPoint:])

# group training races into sets/batches
batchSize = 5
split_points = range(batchSize, len(trainRaces), batchSize)
batchedTrainRaces = np.split(trainRaces, split_points)
# print(len(batchedTrainRaces))

# sums =0
# for i in range(len(batchedTrainRaces)) :
#     for ii in range(len(batchedTrainRaces[i])) :
#         sums += len(batchedTrainRaces[i][ii])
# print(sums)


In [ ]:
# call minimise to optimise
bayesianOptimiser.minimize(np.array(batchedTrainRaces))

# optimal input to be used to update GP model
print(bayesianOptimiser.res['max'])

### Iterate through Race Profiles (of scheduled / upcoming races)

### Calculate Race Learning Index

In [ ]:

# using trained model...
gp = bayesianOptimiser.gp

horseSD = []

horsesSet = []
# get fitted model
for race in testRaces: 
    for horse in race:
        horsesSet.append(horse)
        
horsesSet = np.array(horsesSet)

horseS = []
fastest = 0
fastestHorse = 0

while len(horsesSet) != 0 :
    for i in range(len(horsesSet)) :
        if horsesSet[i][0] > fastest :
            fastestHorse = i
            fastest = horsesSet[i][0]
            
    horseS.append(horsesSet[fastestHorse])
    horsesSet = np.delete(horsesSet,fastestHorse,0)
    fastestHorse = 0
    fastest =0
horseS =np.array(horseS)

for horse in horseS :
    horse_timing, sigma = gp.predict(horse[1:], return_std=True)
    horseSD.append([abs(horse[0] - horse_timing),sigma, horse[0],  horse_timing])

# predict test set
for sd in horseSD :
    print(sd)
    



### Output Race with Max. Index

In [ ]:
actual = []
diff = []
for h in range(len(horseSD)) :
    actual.insert(h,horseSD[h][2])
    diff.insert(h,horseSD[h][0])

plt.plot(actual, diff)
    



## Predict a Race and Use Decision Function to Determine Whether to Bet

## Plot Results with Different Params